# MoBioFP - Semantic Segmentation (U-Net)

## Set Environment Variables

In [ ]:
%env SM_FRAMEWORK=tf.keras

## Import Python Libraries

In [ ]:
import cv2
import os
import math
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import fingerprint_enhancer
import fingerprint_feature_extractor

from rembg import remove
# from google.colab import drive
from keras.models import Model
from keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    concatenate,
    Conv2DTranspose,
    BatchNormalization,
    Activation,
    Dropout,
)

## Global Costants

In [ ]:
DATA_DIR = "../data"
PROCESSED_DIR = DATA_DIR + '/processed'
# IMAGE_PATH = DATA_DIR + '/raw/samples/4_i_1_w_4.jpg'
# IMAGE_PATH = DATA_DIR + '/raw/samples/37_i_1_n_7.jpg'
# IMAGE_PATH = DATA_DIR + '/raw/samples/1_i_1_n_1.jpg'
IMAGE_PATH = DATA_DIR + '/raw/samples/1_o_1_n_1.jpg'
# IMAGE_PATH = DATA_DIR + '/raw/samples/1_i_1_w_1.jpg'
# IMAGE_PATH = DATA_DIR + '/raw/samples/1_o_1_w_1.jpg'
MODELS_DIR = "../models"
MODEL_CHECKPOINT_PATH = MODELS_DIR + "/unet297-v1/arm64/weights/best.h5"

## Define Custom U-Net Model

In [ ]:
# Function to create convolutional block
def conv_block(tensor, nfilters, size=3, padding="same", initializer="he_normal"):
    x = Conv2D(
        filters=nfilters,
        kernel_size=(size, size),
        padding=padding,
        kernel_initializer=initializer,
    )(tensor)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(
        filters=nfilters,
        kernel_size=(size, size),
        padding=padding,
        kernel_initializer=initializer,
    )(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

# Function to create encoder block
def encoder_block(inputs, n_filters):
    conv = conv_block(inputs, n_filters)
    pool = MaxPooling2D(pool_size=(2, 2))(conv)

    return pool, conv

# Function to create decoder block
def decoder_block(inputs, conv_output, n_filters):
    deconv = Conv2DTranspose(
        n_filters, kernel_size=(3, 3), strides=(2, 2), padding="same"
    )(inputs)
    concat = concatenate([deconv, conv_output], axis=3)
    conv = conv_block(concat, n_filters)

    return conv

# Function to create U-Net model
def create_unet(input_shape, n_filters=64):
    inputs = Input(shape=input_shape, name="image_input")
    p1, c1 = encoder_block(inputs, n_filters)
    p2, c2 = encoder_block(p1, n_filters * 2)
    p3, c3 = encoder_block(p2, n_filters * 4)
    p4, c4 = encoder_block(p3, n_filters * 8)
    p4 = Dropout(0.5)(p4)
    c5 = conv_block(p4, n_filters * 16)
    c5 = Dropout(0.5)(c5)
    d6 = decoder_block(c5, c4, n_filters * 8)
    d6 = Dropout(0.5)(d6)
    d7 = decoder_block(d6, c3, n_filters * 4)
    d7 = Dropout(0.5)(d7)
    d8 = decoder_block(d7, c2, n_filters * 2)
    d9 = decoder_block(d8, c1, n_filters)
    outputs = Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid")(d9)
    model = Model(inputs=inputs, outputs=outputs, name="Unet")

    return model

## Load a U-Net pre-trained Model

In [ ]:
model = create_unet(input_shape=(256, 256, 3), n_filters=64)
model.summary()
model.load_weights(MODEL_CHECKPOINT_PATH)

## Read Random Input Image

In [ ]:

image = cv2.imread(IMAGE_PATH)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.title(f'Original image: {IMAGE_PATH}')
plt.axis('off')
plt.show()

## Fingertip Semantic Segmentation

In [ ]:
# Preprocess image
image_resized = cv2.resize(image, (256, 256), interpolation=cv2.INTER_AREA) / 255.0
image_input = np.expand_dims(image_resized, axis=0)

# Predict the mask
predicted_mask = (model.predict(image_input) > 0.5).astype(np.uint8).reshape(256, 256)

# Resize the mask to the original image size
predicted_mask = cv2.resize(predicted_mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_LINEAR)

plt.figure(figsize=(15, 15))
plt.subplot(1, 3, 1)
plt.imshow(image)
plt.title("Original Image")
plt.axis("off")
plt.subplot(1, 3, 2)
plt.imshow(predicted_mask, cmap="gray")
plt.title("Predicted Fingertip Mask")
plt.axis("off")
plt.subplot(1, 3, 3)
plt.imshow(image)
plt.imshow(predicted_mask, cmap="jet", alpha=0.5)
plt.title("Original Image with Predicted Mask")
plt.tight_layout()
plt.show()

## Extract Fingertip ROI

In [ ]:
def extract_roi(mask: np.ndarray, factor: float = 1.10) -> tuple[int, int, int, int]:
    """
    Extract ROI from a binary mask
    Args:
        mask: Binary mask.
        factor: Factor to increase the size of the ROI.
    Returns:
        Tuple with four coordinates representing the bounding box rectangle.
    """
    cnts, _ = cv2.findContours(
        mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    cnt = max(cnts, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(cnt)

    # Adjust the size of the ROI
    w_new = int(w * factor)
    h_new = int(h * factor)
    x_new = max(0, x - (w_new - w) // 2)
    y_new = max(0, y - (h_new - h) // 2)

    return (x_new, y_new, w_new, h_new)

# Extract ROI (fingertip)


In [ ]:
(x, y, w, h) = extract_roi(predicted_mask)

# Create a rectangle patch for the ROI
roi_rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor="r", facecolor="none")

fig, axes = plt.subplots(1, 2, figsize=(20, 7))
axes[0].imshow(image)
axes[0].add_patch(roi_rect)
axes[0].set_title("Original Image: Fingertip ROI")

# Crop the ROI from the original image
fingertip = image[y : y + h, x : x + w]
axes[1].imshow(fingertip)
axes[1].set_title("Segmented Fingertip")

plt.tight_layout()
plt.show()

## Fingertip Background Removal

In [ ]:
fingertip_mask = remove(fingertip, only_mask=True)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
fingertip_mask = cv2.morphologyEx(fingertip_mask, cv2.MORPH_OPEN, kernel, iterations=2)
fingertip_mask = cv2.GaussianBlur(fingertip_mask, (5, 5), sigmaX=2, sigmaY=2, borderType=cv2.BORDER_DEFAULT)
fingertip_mask = np.where(fingertip_mask < 127, 0, 255).astype(np.uint8)

plt.figure(figsize=(10, 10))
plt.subplot(1, 3, 1)
plt.imshow(fingertip, cmap="gray")
plt.title('Original fingertip')
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(fingertip_mask, cmap="gray")
plt.title("Binary Mask: Baground Removal")
plt.axis('off')
plt.subplot(1, 3, 3)
plt.imshow(cv2.bitwise_and(fingertip, fingertip, mask=fingertip_mask))
plt.title("Fingertip with background removed")
plt.axis('off')
plt.tight_layout()
plt.show()

fingertip = cv2.bitwise_and(fingertip, fingertip, mask=fingertip_mask)

## Fingertip Orientation

In [ ]:
# Convert the fingertip to grayscale
fingertip_gray = cv2.cvtColor(fingertip, cv2.COLOR_RGB2GRAY)

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.imshow(fingertip_gray, cmap="gray")
plt.title("Fingertip Grayscale:" + str(fingertip_gray.shape))

plt.subplot(1, 2, 2)
hist = cv2.calcHist([fingertip_gray], [0], None, [256], [0, 256])
hist /= hist.sum()
plt.plot(hist)
plt.xlim([0, 256])
plt.title("Histogram")
plt.xlabel("Pixel Value")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

In [ ]:
def getOrientation(pts, img):
    sz = len(pts)
    data_pts = np.empty((sz, 2), dtype=np.float64)
    for i in range(data_pts.shape[0]):
        data_pts[i,0] = pts[i,0,0]
        data_pts[i,1] = pts[i,0,1]
    # Perform PCA analysis
    mean = np.empty((0))
    mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean)
    angle = math.atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians

    return angle

In [ ]:
# Find contours in the fingertip grayscale image
contours, _ = cv2.findContours(fingertip_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

# Sort the contours by area
contours = sorted(contours, key=cv2.contourArea, reverse=True)

# Find the contour with the largest area
largest_contour = contours[0]

# Find the orientation of the largest contour
angle = getOrientation(largest_contour, fingertip_gray)

# If the angle is negative, make it positive
if angle < 0:
    angle = 90 + angle

# Rotate the fingertip
fingertip_rotated = imutils.rotate_bound(fingertip, angle)
fingertip_gray_rotated = imutils.rotate_bound(fingertip_gray, angle)
fingertip_mask_rotated = imutils.rotate_bound(fingertip_mask, angle)

plt.figure(figsize=(10, 10))
plt.subplot(1, 3, 1)
plt.imshow(fingertip_rotated)
plt.title('Original fingertip rotated')
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(fingertip_gray_rotated, cmap="gray")
plt.title("Gray fingertip rotated")
plt.axis('off')
plt.subplot(1, 3, 3)
plt.imshow(fingertip_mask_rotated, cmap="gray")
plt.title("Binary Mask rotated")
plt.axis('off')
plt.tight_layout()
plt.show()

## Fingertip Enhancement

In [ ]:
# Normalize the image
normalized = cv2.normalize(fingertip_gray_rotated, None, 0, 255, cv2.NORM_MINMAX)

# Apply bilateral filter
bilateral = cv2.bilateralFilter(normalized, 7, 50, 50)

# Apply CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)).apply(bilateral)

plt.figure(figsize=(10, 10))
plt.subplot(1, 4, 1)
plt.imshow(fingertip_gray_rotated, cmap="gray")
plt.title('Original fingertip rotated')
plt.axis('off')
plt.subplot(1, 4, 2)
plt.imshow(normalized, cmap="gray")
plt.title("Normalized")
plt.axis('off')
plt.subplot(1, 4, 3)
plt.imshow(bilateral, cmap="gray")
plt.title("Bilateral")
plt.axis('off')
plt.subplot(1, 4, 4)
plt.imshow(clahe, cmap="gray")
plt.title("CLAHE")
plt.axis('off')
plt.tight_layout()
plt.show()

fingertip_gray_rotated = clahe

## Adaptive Thresholding

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.imshow(fingertip_gray_rotated, cmap="gray")
plt.title("Fingertip Grayscale:" + str(fingertip_gray_rotated.shape))

plt.subplot(1, 2, 2)
hist, bins = np.histogram(fingertip_gray_rotated.flatten(),256,[0,256])
cdf = hist.cumsum()
cdf_normalized = cdf * hist.max()/ cdf.max()

plt.plot(cdf_normalized, color = 'b')
plt.hist(fingertip_gray_rotated.flatten(),256,[0,256], color = 'r')
plt.xlim([0,256])
plt.legend(('cdf','histogram'), loc = 'upper left')
plt.title("Histogram")
plt.tight_layout()
plt.show()

In [ ]:
thresh_mean = cv2.adaptiveThreshold(fingertip_gray_rotated, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(fingertip_gray_rotated, cmap="gray")
plt.title('Original fingertip rotated')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(thresh_mean, cmap="gray")
plt.title("Adaptive threshold mean")
plt.axis('off')
plt.tight_layout()
plt.show()

thresh = thresh_mean

## Compute Gabor Filters (TO BE REVIEWED)

In [ ]:
_sigma_conv = (3.0/2.0)/((6*math.log(10))**0.5)
# sigma is adjusted according to the ridge period, so that the filter does not contain more than three effective peaks
def _gabor_sigma(ridge_period):
    return _sigma_conv * ridge_period

def _gabor_size(ridge_period):
    p = int(round(ridge_period * 2 + 1))
    if p % 2 == 0:
        p += 1
    return (p, p)

def gabor_kernel(period, orientation):
    f = cv2.getGaborKernel(_gabor_size(period), _gabor_sigma(period), np.pi/2 - orientation, period, gamma = 1, psi = 0)
    f /= f.sum()
    f -= f.mean()
    return f

def from_amt_to_scan(image):
    """heavily from https://colab.research.google.com/drive/1u5X8Vg9nXWPEDFFtUwbkdbQxBh4hba_M"""

    fingerprint = image

    # Calculate the local gradient (using Sobel filters)
    gx, gy = cv2.Sobel(fingerprint, cv2.CV_32F, 1, 0), cv2.Sobel(fingerprint, cv2.CV_32F, 0, 1)

    # Calculate the magnitude of the gradient for each pixel
    gx2, gy2 = gx**2, gy**2

    W = (29, 29) # (23, 23)
    gxx = cv2.boxFilter(gx2, -1, W, normalize = False)
    gyy = cv2.boxFilter(gy2, -1, W, normalize = False)
    gxy = cv2.boxFilter(gx * gy, -1, W, normalize = False)
    gxx_gyy = gxx - gyy
    gxy2 = 2 * gxy

    orientations = (cv2.phase(gxx_gyy, -gxy2) + np.pi) / 2 # '-' to adjust for y axis direction

    # _region = fingerprint[10:90,80:130]
    center_h = fingerprint.shape[0]//3
    center_w = fingerprint.shape[1]//2
    #_region = fingerprint[center_h-40:center_h+40, center_w-25:center_w+25]
    region = fingerprint[center_h-40:center_h+40, center_w+10:center_w+60]

    # before computing the x-signature, the region is smoothed to reduce noise
    smoothed = cv2.blur(region, (5,5), -1)
    xs = np.sum(smoothed, 1) # the x-signature of the region

    # Find the indices of the x-signature local maxima
    local_maxima = np.nonzero(np.r_[False, xs[1:] > xs[:-1]] & np.r_[xs[:-1] >= xs[1:], False])[0]

    # Calculate all the distances between consecutive peaks
    distances = local_maxima[1:] - local_maxima[:-1]

    # Estimate the ridge line period as the average of the above distances
    ridge_period = np.average(distances)

    # Create the filter bank
    or_count = 8
    gabor_bank = [gabor_kernel(ridge_period, o) for o in np.arange(0, np.pi, np.pi/or_count)]

    # Filter the whole image with each filter
    # Note that the negative image is actually used, to have white ridges on a black background as a result!!
    nf = 255-fingerprint
    all_filtered = np.array([cv2.filter2D(nf, cv2.CV_32F, f) for f in gabor_bank])

    y_coords, x_coords = np.indices(fingerprint.shape)
    # For each pixel, find the index of the closest orientation in the gabor bank
    orientation_idx = np.round(((orientations % np.pi) / np.pi) * or_count).astype(np.int32) % or_count
    # Take the corresponding convolution result for each pixel, to assemble the final result
    filtered = all_filtered[orientation_idx, y_coords, x_coords]
    # Convert to gray scale and apply the mask
    enhanced = np.clip(filtered, 0, 255).astype(np.uint8)

    return enhanced

In [ ]:
thresh_resized = imutils.resize(thresh, width=400)
# GABOR_IMAGE_SHAPE = (400, 840)
# thresh_resized = cv2.resize(thresh, GABOR_IMAGE_SHAPE, interpolation=cv2.INTER_AREA)
enhanced_gabor = from_amt_to_scan(thresh_resized)

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(thresh_resized, cmap="gray")
plt.title('Adaptive threshold resized: ' + str(thresh_resized.shape))
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(enhanced_gabor, cmap="gray")
plt.title("Fingerprint Enhanced with Gabor filter")
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
enhanced_fingerprint = fingerprint_enhancer.enhance_Fingerprint(enhanced_gabor)

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(enhanced_gabor, cmap="gray")
plt.title('Enhanced Gabor')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(enhanced_fingerprint, cmap="gray")
plt.title("Enhanced Fingerprint")
plt.axis('off')
plt.tight_layout()
plt.show()

## Fingerprint Feature Extractor

In [ ]:
terminations, bifurcations = fingerprint_feature_extractor.extract_minutiae_features(
    enhanced_fingerprint,
    spuriousMinutiaeThresh=25,
    showResult=False,
    saveResult=True
)
print(f'Terminations: {len(terminations)}')
print(f'Bifurcations: {len(bifurcations)}')